In [1]:
from math import log
import numpy as np
import operator

In [2]:
def loaddata ():
    dataSet = [[0, 0,0,0,0,0, 'yes'],
               [1, 0,1,0,0,0,'yes'],
               [1, 0,0,0,0,0,'yes'],
               [0, 0,1,0,0,0,'yes'],
               [2, 0,0,0,0,0,'yes'],
               [0, 1,0,0,1,1,'yes'],
               [1, 1,0,1,1,1,'yes'],
               [1, 1,0,0,1,0, 'yes'],
               [1, 1,1,1,1,0,'no'],
               [0, 2,2,0,2,1,'no'],
               [2, 2,2,2,2,0,'no'],
               [2, 0,0,2,2,1,'no'],
               [0, 1,0,1,0,0, 'no'],
               [2, 1,1,1,0,0,'no'],
               [1, 1,0,0,1,1,'no'],
               [2, 0,0,2,2,0,'no'],
               [0, 0,1,1,1,0,'no']]
    feature_name = ['a1','a2','a3','a4','a5','a6']
    return dataSet, feature_name

In [3]:
def entropy(dataSet):
    """计算信息熵"""
    #数据集条数
    m = len(dataSet)
    #保存所有的类别及属于该类别的样本数
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys(): 
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    #保存熵值
    e=0.0
    for label in labelCounts.keys():
        e -=labelCounts[label]*1.0/m *log(labelCounts[label]*1.0/m,2)
    return e

In [4]:
def splitDataSet(dataSet, axis, value):
    """给定特征和特征值划分数据集dataSet"""
    # axis对应的是特征的索引;
    retDataSet = []
    #遍历数据集
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            #上面两行是拿到除了当前特征的其他列
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [5]:
def chooseBestFeature(dataSet):
    """据信息增益选择最好的特征"""
    n = len(dataSet[0]) - 1 #特征的数目
    #计数整个数据集的熵
    baseEntropy = entropy(dataSet)
    bestInfoGain = 0.0; bestFeature = -1
    #遍历每个特征
    for i in range(n):  
        #获取当前特征i的所有可能取值
        featList = [example[i] for example in dataSet] #取出每个特征的值
        uniqueVals = set(featList)#特征所具有的值的集合
        newEntropy = 0.0
        #遍历特征i的每一个可能的取值
        for value in uniqueVals:
            #按特征i的value值进行数据集的划分
            subDataSet = splitDataSet(dataSet, i, value)
            #补充计算条件熵的代码
            newEntropy +=len(subDataSet)*1.0/len(dataSet)*entropy(subDataSet) #这里计算每个子集的条件熵，并相加
        #计算信息增益  
        infoGain = baseEntropy - newEntropy  
        #保存当前最大的信息增益及对应的特征
        if (infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

In [6]:
def classVote(classList):
    """获取出现次数最多的类别名称"""
    #定义字典，保存每个标签对应的个数 
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): 
            classCount[vote] = 0
        classCount[vote] += 1
     #排序找到数目最多的类别
     #dict.item()将key和value组成一个元组，并把这些元组放在列表中返回。
    #operator.itemgetter(1)按照第二个元素的次序对元组进行排序
    #reverse=True是逆序，即按照从大到小的顺序排列
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [7]:
def trainTree(dataSet,feature_name):
    classList = [example[-1] for example in dataSet] #拿到所有的标签
    #所有类别都一致
    if classList.count(classList[0]) == len(classList): 
        return classList[0] 
    #数据集中没有特征
    if len(dataSet[0]) == 1: 
        return classVote(classList)
    #选择最优划分特征:返回最优特征索引
    bestFeat = chooseBestFeature(dataSet)
    #得到最优特征的标签
    bestFeatName = feature_name[bestFeat]
    myTree = {bestFeatName:{}}
    del feature_name[bestFeat]
    featValues = [example[bestFeat] for example in dataSet] #拿到该特征的所有值
    uniqueVals = set(featValues)#去重
    #遍历uniqueVals中的每个值，生成相应的分支
    for value in uniqueVals:
        sub_feature_name = feature_name[:]
        # 生成在dataSet中bestFeat取值为value的子集；
        sub_dataset = splitDataSet(dataSet,bestFeat,value)
        # 根据得到的子集，生成决策树
        myTree[bestFeatName][value] = trainTree(sub_dataset,sub_feature_name)
    return myTree

In [8]:
myDat,feature_name = loaddata()
myTree = trainTree(myDat,feature_name)
print(myTree)

{'a4': {0: {'a2': {0: 'yes', 1: {'a1': {0: 'yes', 1: {'a6': {0: 'yes', 1: 'no'}}}}, 2: 'no'}}, 1: {'a6': {0: 'no', 1: 'yes'}}, 2: 'no'}}
